<H1> CLTV Prediction with MLflow</H1>

We are going to build simple machine learning models that predicts our customers lifetime value and compare their performances, now using MLflow.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report,precision_recall_fscore_support,accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

import mlflow

<a name=1> <h1> 1. Feature Engineering </h2></a>

In [2]:
# get the data
import pickle

with open("tx_cluster.pkl", "rb") as f:
    tx_cluster = pickle.load(f)

tx_cluster.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment,m6_Revenue,LTVCluster
0,17850.0,301,0,312,1,5288.63,1,2,Low-Value,0.00,0
1,17602.0,1,3,565,1,5050.77,1,5,High-Value,128.75,0
2,17509.0,57,2,369,1,6100.74,1,4,Mid-Value,0.00,0
3,13093.0,266,0,170,0,7741.47,1,1,Low-Value,262.55,0
4,17809.0,15,3,64,0,4627.62,1,4,Mid-Value,276.72,0


In [3]:
#convert categorical columns to numerical
tx_class = pd.get_dummies(tx_cluster) #There is only one categorical variable segment
tx_class.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,m6_Revenue,LTVCluster,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,17850.0,301,0,312,1,5288.63,1,2,0.00,0,False,True,False
1,17602.0,1,3,565,1,5050.77,1,5,128.75,0,True,False,False
2,17509.0,57,2,369,1,6100.74,1,4,0.00,0,False,False,True
3,13093.0,266,0,170,0,7741.47,1,1,262.55,0,False,True,False
4,17809.0,15,3,64,0,4627.62,1,4,276.72,0,False,False,True


In [4]:
tx_class.describe()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,m6_Revenue,LTVCluster
count,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000
mean,15561.356010,91.613811,2.098977,84.348849,0.113555,1289.904252,0.057545,2.270077,513.677960,0.221995
std,1575.088247,100.383464,1.055228,152.686136,0.332276,2117.764962,0.235097,1.273618,873.629228,0.483366
min,12346.000000,0.000000,0.000000,1.000000,0.000000,-4287.630000,0.000000,0.000000,-4287.630000,0.000000
25%,14209.250000,16.000000,1.000000,16.250000,0.000000,279.230000,0.000000,1.000000,0.000000,0.000000
50%,15569.500000,49.000000,2.000000,40.000000,0.000000,618.040000,0.000000,2.000000,205.305000,0.000000
75%,16911.500000,144.000000,3.000000,98.000000,0.000000,1483.497500,0.000000,3.000000,626.360000,0.000000
max,18287.000000,373.000000,3.000000,5128.000000,3.000000,57120.910000,2.000000,8.000000,7387.220000,2.000000


In [5]:
#calculate and show correlations
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

LTVCluster            1.000000
m6_Revenue            0.890176
Revenue               0.673601
RevenueCluster        0.603433
Segment_High-Value    0.511627
FrequencyCluster      0.470566
Frequency             0.447167
OverallScore          0.419524
RecencyCluster        0.223734
Segment_Mid-Value     0.037707
CustomerID           -0.044819
Recency              -0.224075
Segment_Low-Value    -0.234132
Name: LTVCluster, dtype: float64

In [6]:
#create X and y, X will be feature set and y is the label - LTV
X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = tx_class['LTVCluster']

#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

## Models

Since our LTV Clusters are 3 types, high LTV, mid LTV and low LTV; we will perform multi class classification.

In [7]:
mlflow.set_experiment("CLTV_testsize005")
mlflow.sklearn.autolog(disable=True)

### 1. Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

basemodelname = "Logit_test"
with mlflow.start_run(run_name=basemodelname):
    params = {
        "penalty": None,
        "class_weight": 'balanced'}
    parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
    modelname=f"{basemodelname}_{parsuf}"
    mlflow.set_tag("model_name", modelname)
    mlflow.log_params(params)

    ltv_logreg = LogisticRegression(
        penalty=params['penalty'],
        class_weight=params['class_weight'],
        max_iter=1000
    ).fit(X_train, y_train)

    acc_train = ltv_logreg.score(X_train, y_train)
    acc_test = ltv_logreg.score(X_test[X_train.columns], y_test)

    print(f"Modelname: {modelname}")
    print('Accuracy of Logit classifier on training set: {:.2f}'.format(acc_train))
    print('Accuracy of Logit classifier on test set: {:.2f}'.format(acc_test))

    y_pred = ltv_logreg.predict(X_test)
    # clfreport = classification_report(y_test, y_pred)
    # print(clfreport)
    test_prf1s = precision_recall_fscore_support(y_test, y_pred)
    # print(f"precision:{prf1s[0]}\nrecall:{prf1s[1]}\nf1-score:{prf1s[2]}\naccuracy:{acc}")

    # log the skill metrics
    mlflow.log_metric('train_acc', acc_train)
    mlflow.log_metric('test_acc', acc_test)
    mlflow.log_metric('test_macroavg_f1', np.mean(test_prf1s[2]))

    # log the model as an artifact to enable later use
    mlflow.sklearn.log_model(ltv_logreg, "ltv_logreg")

Modelname: Logit_test_penaltyNone_classweightbalanced
Accuracy of Logit classifier on training set: 0.87
Accuracy of Logit classifier on test set: 0.87


C:\Users\onur.kerimoglu\Projects\Repos\ML-sandbox\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### 2. XGBoost: testing

In [9]:
import xgboost as xgb

basemodelname = "xgboost_test"
with mlflow.start_run(run_name=basemodelname):
    params = {
        "max_depth": 4,
        "learning_rate":0.05}
    parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
    modelname=f"{basemodelname}_{parsuf}"
    mlflow.set_tag("model_name", modelname)
    mlflow.log_params(params)
    
    ltv_xgb = xgb.XGBClassifier(
        max_depth=params['max_depth'], 
        learning_rate=params['learning_rate'],
        n_jobs=-1
    ).fit(X_train, y_train)
    
    acc_train = ltv_xgb.score(X_train, y_train)
    acc_test = ltv_xgb.score(X_test[X_train.columns], y_test)
    
    print(f"Modelname: {modelname}")
    print('Accuracy of XGB classifier on training set: {:.2f}'.format(acc_train))
    print('Accuracy of XGB classifier on test set: {:.2f}'.format(acc_test))
    
    y_pred = ltv_xgb.predict(X_test)
    # clfreport = classification_report(y_test, y_pred)
    # print(clfreport)
    test_prf1s = precision_recall_fscore_support(y_test, y_pred)
    # print(f"precision:{prf1s[0]}\nrecall:{prf1s[1]}\nf1-score:{prf1s[2]}\naccuracy:{acc}")
    
    # log the skill metrics
    mlflow.log_metric('train_acc', acc_train)
    mlflow.log_metric('test_acc', acc_test)
    mlflow.log_metric('test_macroavg_f1', np.mean(test_prf1s[2]))
    
    # log the model as an artifact to enable later use
    mlflow.xgboost.log_model(ltv_xgb, "ltv_xgb")

Modelname: xgboost_test_maxdepth4_learningrate005
Accuracy of XGB classifier on training set: 0.93
Accuracy of XGB classifier on test set: 0.89


### 2b. XGboost: parameter optimization

In [10]:
basemodelname = "xgboost_paropt"
with mlflow.start_run(run_name=basemodelname):
    params_list = {
        "max_depth": [4, 5],
        "learning_rate":[0.05, 0.1]}
    
    run_i = 0
    for max_depth in params_list['max_depth']:
        for learning_rate in params_list['learning_rate']:
            run_i += 1
            params = {
                "max_depth": max_depth,
                "learning_rate": learning_rate}
            parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
                
            with mlflow.start_run(run_name=parsuf, nested=True) as subrun_i:
                    
                modelname=f"{basemodelname}_{parsuf}"
                mlflow.set_tag("model_name", modelname)
                mlflow.log_params(params)

                ltv_xgb = xgb.XGBClassifier(
                    max_depth=params['max_depth'], 
                    learning_rate=params['learning_rate'],
                    n_jobs=-1
                ).fit(X_train, y_train)

                acc_train = ltv_xgb.score(X_train, y_train)
                acc_test = ltv_xgb.score(X_test[X_train.columns], y_test)

                print(f"Modelname: {modelname}")
                print('Accuracy of XGB classifier on training set: {:.2f}'.format(acc_train))
                print('Accuracy of XGB classifier on test set: {:.2f}'.format(acc_test))

                y_pred = ltv_xgb.predict(X_test)
                # clfreport = classification_report(y_test, y_pred)
                # print(clfreport)
                test_prf1s = precision_recall_fscore_support(y_test, y_pred)
                # print(f"precision:{prf1s[0]}\nrecall:{prf1s[1]}\nf1-score:{prf1s[2]}\naccuracy:{acc}")

                # log the skill metrics
                mlflow.log_metric('train_acc', acc_train)
                mlflow.log_metric('test_acc', acc_test)
                mlflow.log_metric('test_macroavg_f1', np.mean(test_prf1s[2]))

                # log the model as an artifact to enable later use
                mlflow.xgboost.log_model(ltv_xgb, "ltv_xgb")

Modelname: xgboost_paropt_maxdepth4_learningrate005
Accuracy of XGB classifier on training set: 0.93
Accuracy of XGB classifier on test set: 0.89
Modelname: xgboost_paropt_maxdepth4_learningrate01
Accuracy of XGB classifier on training set: 0.94
Accuracy of XGB classifier on test set: 0.89
Modelname: xgboost_paropt_maxdepth5_learningrate005
Accuracy of XGB classifier on training set: 0.94
Accuracy of XGB classifier on test set: 0.89
Modelname: xgboost_paropt_maxdepth5_learningrate01
Accuracy of XGB classifier on training set: 0.96
Accuracy of XGB classifier on test set: 0.89


## Test split: 0.2

In [11]:
#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlflow.set_experiment("CLTV_testsize02")
mlflow.sklearn.autolog(disable=True)

2023/07/26 15:16:02 INFO mlflow.tracking.fluent: Experiment with name 'CLTV_testsize02' does not exist. Creating a new experiment.


In [12]:
from sklearn.linear_model import LogisticRegression

basemodelname = "TestSplit02_Logit_test"
with mlflow.start_run(run_name=basemodelname):
    params = {
        "penalty": None,
        "class_weight": 'balanced'}
    parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
    modelname=f"{basemodelname}_{parsuf}"
    mlflow.set_tag("model_name", modelname)
    mlflow.log_params(params)

    ltv_logit = LogisticRegression(
        penalty=params['penalty'],
        class_weight=params['class_weight'],
        max_iter=1000
    ).fit(X_train, y_train)

    acc_train = ltv_logit.score(X_train, y_train)
    acc_test = ltv_logit.score(X_test[X_train.columns], y_test)

    print(f"Modelname: {modelname}")
    print('Accuracy of Logit classifier on training set: {:.2f}'.format(acc_train))
    print('Accuracy of Logit classifier on test set: {:.2f}'.format(acc_test))

    y_pred = ltv_logreg.predict(X_test)
    # clfreport = classification_report(y_test, y_pred)
    # print(clfreport)
    test_prf1s = precision_recall_fscore_support(y_test, y_pred)
    # print(f"precision:{prf1s[0]}\nrecall:{prf1s[1]}\nf1-score:{prf1s[2]}\naccuracy:{acc}")

    # log the skill metrics
    mlflow.log_metric('train_acc', acc_train)
    mlflow.log_metric('test_acc', acc_test)
    mlflow.log_metric('test_macroavg_f1', np.mean(test_prf1s[2]))

    # log the model as an artifact to enable later use
    mlflow.sklearn.log_model(ltv_logit, "ltv_logit")

Modelname: TestSplit02_Logit_test_penaltyNone_classweightbalanced
Accuracy of Logit classifier on training set: 0.88
Accuracy of Logit classifier on test set: 0.87


## Use a model that we saved earlier

In [13]:
if False:
    import mlflow
    import pandas as pd
    import pickle
    import xgboost
    from sklearn.model_selection import train_test_split

    with open("tx_class.pkl", "rb") as f:
        tx_cluster = pickle.load(f)

    #convert categorical columns to numerical
    tx_class = pd.get_dummies(tx_cluster) #There is only one categorical variable segment
    tx_class.head()

    #create X and y, X will be feature set and y is the label - LTV
    X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
    y = tx_class['LTVCluster']

    #split training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

    ltv_xgb = mlflow.xgboost.load_model("runs:/f9f153aa679d4a8697cf1b23a0d479ac/ltv_xgb")

    acc_train = ltv_xgb.score(X_train, y_train)
    acc_test = ltv_xgb.score(X_test[X_train.columns], y_test)

    print('Accuracy of XGB classifier on training set: {:.2f}'.format(acc_train))
    print('Accuracy of XGB classifier on test set: {:.2f}'.format(acc_test))